In [155]:
import org.apache.spark.sql.functions.{min, max}
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import scala.util.Random
import org.apache.spark.sql.functions.lit
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{Dataset, Row}

In [2]:
val ratingsTrain = "../datasets/ua.base"
val ratingsTest = "../datasets/ua.test"

ratingsTrain = ../datasets/ua.base
ratingsTest = ../datasets/ua.test


../datasets/ua.test

In [3]:
val colNames = Array("userId", "movieId", "rating", "ts")
val dfTrain = spark.read.options(Map("header" -> "false", "inferSchema" -> "true", "sep" -> "\t")).csv(ratingsTrain).toDF(colNames: _*)
val dfTest = spark.read.options(Map("header" -> "false", "inferSchema" -> "true", "sep" -> "\t")).csv(ratingsTest).toDF(colNames: _*)

colNames = Array(userId, movieId, rating, ts)
dfTrain = [userId: int, movieId: int ... 2 more fields]
dfTest = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

### Parameters configuration

In [4]:
val ranks = Seq(10, 20)
val regParams = Seq(1, 0.001)
val alphas = Seq(1, 0.001)

ranks = List(10, 20)
regParams = List(1.0, 0.001)
alphas = List(1.0, 0.001)


List(1.0, 0.001)

In [5]:
val alsModels = for {
    rank <- ranks
    regParam <- regParams
    alpha <- alphas
} yield (Map( "rank" -> rank, "regParam" -> regParam, "alpha" -> alpha), new ALS()
            .setRank(rank)
            .setUserCol("userId")
            .setItemCol("movieId")
            .setImplicitPrefs(true)
            .fit(dfTrain)
            )

alsModels = List((Map(rank -> 10, regParam -> 1.0, alpha -> 1.0),als_e5770c05bf42), (Map(rank -> 10, regParam -> 1.0, alpha -> 0.001),als_bb3ca58ca8af), (Map(rank -> 10, regParam -> 0.001, alpha -> 1.0),als_b010de0eed11), (Map(rank -> 10, regParam -> 0.001, alpha -> 0.001),als_9c55277f2950), (Map(rank -> 20, regParam -> 1.0, alpha -> 1.0),als_dc40fd4edac1), (Map(rank -> 20, regParam -> 1.0, alpha -> 0.001),als_153792232371), (Map(rank -> 20, regParam -> 0.001, alpha -> 1.0),als_4fc95f3f0123), (Map(rank -> 20, regParam -> 0.001, alpha -> 0.001),als_00a6e7a6bd25))


List((Map(rank -> 10, regParam -> 1.0, alpha -> 1.0),als_e5770c05bf42), (Map(rank -> 10, regParam -> 1.0, alpha -> 0.001),als_bb3ca58ca8af), (Map(rank -> 10, regParam -> 0.001, alpha -> 1.0),als_b010de0eed11), (Map(rank -> 10, regParam -> 0.001, alpha -> 0.001),als_9c55277f2950), (Map(rank -> 20, regParam -> 1.0, alpha -> 1.0),als_dc40fd4edac1), (Map(rank -> 20, regParam -> 1.0, alpha -> 0.001),als_153792232371), (Map(rank -> 20, regParam -> 0.001, alpha -> 1.0),als_4fc95f3f0123), (Map(rank -> 20, regParam -> 0.001, alpha -> 0.001),als_00a6e7a6bd25))

In [6]:
alsModels.foreach { _._2.setColdStartStrategy("drop") }

In [111]:
val fullDataset = dfTrain.union(dfTest)

fullDataset = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [112]:
val users = fullDataset.select("userId").distinct()

users = [userId: int]


[userId: int]

In [113]:
val movies = fullDataset.select("movieId").distinct

movies = [movieId: int]


[movieId: int]

In [119]:
val userAndMovies = for {
    user <- users.collect
    movie <- movies.collect
} yield (user.getInt(0), movie.getInt(0))

[Stage 5003:=================================>                  (128 + 6) / 200]

userAndMovies = Array((148,148), (148,471), (148,496), (148,463), (148,833), (148,1088), (148,1238), (148,1342), (148,1580), (148,1591), (148,1645), (148,243), (148,392), (148,540), (148,623), (148,737), (148,858), (148,897), (148,1084), (148,1025), (148,1127), (148,1395), (148,1460), (148,1483), (148,1507), (148,1522), (148,31), (148,516), (148,1139), (148,1143), (148,1270), (148,1303), (148,1322), (148,1339), (148,1352), (148,1618), (148,1650), (148,85), (148,137), (148,251), (148,451), (148,580), (148,808), (148,1265), (148,65), (148,458), (148,879), (148,883), (148,1223), (148,53), (148,255), (148,481), (148,588), (148,799), (148,804), (148,898), (148,970), (148,1157), (148,133), (148,296), (148,472), (148,853), (148,1466), (148,78), (148,322), (148,513), (148,918...


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [122]:
val userAndMoviesDF = sc.parallelize(userAndMovies).toDF("userId", "movieId")

userAndMoviesDF = [userId: int, movieId: int]


[userId: int, movieId: int]

In [161]:
val isPositive = udf { v: Int => if (v >= 0) 1 else 0 }

isPositive = UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))

In [162]:
val usersAndMoviesRatingDF = userAndMoviesDF
    .join(fullDataset, Seq("userId", "movieId"), "left_outer")
    .drop("ts")
    .na
    .fill(-1)
    .cache

usersAndMoviesRatingDF = [userId: int, movieId: int ... 1 more field]


[userId: int, movieId: int ... 1 more field]

In [169]:
val usersAndMoviesFullRatingsDF = usersAndMoviesRatingDF
    .withColumn("rating", isPositive($"rating"))
    .withColumnRenamed("rating", "label")

usersAndMoviesFullRatingsDF = [userId: int, movieId: int ... 1 more field]


[userId: int, movieId: int ... 1 more field]

In [190]:
def track_seens() = {
    import scala.collection.mutable
    val _seen: mutable.Map[Int, Int] = mutable.Map[Int, Int]()
    
    udf { (userId: Int) => _seen
         .put(userId, _seen.getOrElse(userId, 0) + 1)
         .getOrElse(0)
    }

}


track_seens: ()org.apache.spark.sql.expressions.UserDefinedFunction


In [191]:
val seenCount = track_seens()

seenCount = UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))

In [197]:
val usersAndMoviesFalseInteractions = usersAndMoviesFullRatingsDF
    .filter($"label" === 0)
    .withColumn("seen", seenCount($"userId"))
    .filter($"seen" === 0)
    .drop("seen")

usersAndMoviesFalseInteractions = [userId: int, movieId: int ... 1 more field]


[userId: int, movieId: int ... 1 more field]

In [199]:
usersAndMoviesFalseInteractions.printSchema

root
 |-- userId: integer (nullable = false)
 |-- movieId: integer (nullable = false)
 |-- label: integer (nullable = true)



lastException: Throwable = null


In [200]:
dfTest.printSchema

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- ts: integer (nullable = true)



In [220]:
val dfFullTest = usersAndMoviesFalseInteractions
    .union(
        dfTest
            .withColumn("label", lit(1))
            .select("userId", "movieId", "label")
    )

dfFullTest = [userId: int, movieId: int ... 1 more field]


[userId: int, movieId: int ... 1 more field]

In [221]:
dfFullTest.count

[Stage 6897:========================>                               (3 + 4) / 7]

10378

In [7]:
val predictions = alsModels.map { _._2.transform(dfTest) } 

predictions = List([userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields])


List([userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields])

In [40]:
val toInt = udf { v: Float => if (v > 0.5) 1 else 0 }

toInt = UserDefinedFunction(<function1>,IntegerType,Some(List(FloatType)))


lastException: Throwable = null


UserDefinedFunction(<function1>,IntegerType,Some(List(FloatType)))

In [41]:
val predictions2 = predictions.map { (df) => df.withColumn("prediction", toInt(df("prediction"))) }

predictions2 = List([userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields])


List([userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields], [userId: int, movieId: int ... 3 more fields])

In [76]:
val pred = predictions2(0)

pred = [userId: int, movieId: int ... 3 more fields]


[userId: int, movieId: int ... 3 more fields]

In [103]:
val scoreAndLabels = pred.select("rating", "prediction").map { r => (r.getInt(0).toDouble, r.getInt(1).toDouble) }

scoreAndLabels = [_1: double, _2: double]


[_1: double, _2: double]

In [105]:
val metrics = new BinaryClassificationMetrics(scoreAndLabels.rdd)

metrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@6fe3151f


org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@6fe3151f

In [227]:
metrics.areaUnderROC

0.5733430276500435

In [68]:
predictions2(0).map { r => case  } 

Name: Unknown Error
Message: <console>:78: error: Unable to find encoder for type stored in a Dataset.  Primitive types (Int, String, etc) and Product types (case classes) are supported by importing spark.implicits._  Support for serializing other types will be added in future releases.
       predictions2(0).map { r => r(0) }
                           ^

StackTrace: 

In [71]:
val evaluator = new BinaryClassificationEvaluator()

evaluator = binEval_b72bec7f82f7


binEval_b72bec7f82f7

In [74]:
evaluator.explainParams

labelCol: label column name (default: label)
metricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC)
rawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)

In [226]:
predictions(0).printSchema

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- ts: integer (nullable = true)
 |-- prediction: float (nullable = true)

